In [1]:
# Install DuckDuckGo search (Run this in your terminal or notebook cell)
# !pip install -q duckduckgo-search langgraph langchain langchain-openai

from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from duckduckgo_search import DDGS
import os
from dotenv import load_dotenv

# Load API Key
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

c:\Users\user\Documents\working with LLM\benv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@tool
def weather_tool(city: str) -> str:
    """
    Get the current weather for a specific city. 
    Use this when the user asks about weather or temperature.
    """
    # Simulated weather data
    city = city.lower()
    if "lagos" in city:
        return "Lagos: 32°C, Sunny with high humidity."
    elif "london" in city:
        return "London: 15°C, Rainy and cloudy."
    elif "paris" in city:
        return "Paris: 22°C, Partly cloudy."
    else:
        return f"{city.title()}: 25°C, Clear skies (Simulated)."

In [3]:
@tool
def dictionary_tool(word: str) -> str:
    """
    Look up the definition of a word. 
    Use this when the user asks for the meaning or definition of a term.
    """
    definitions = {
        "ephemeral": "Lasting for a very short time.",
        "serendipity": "The occurrence of events by chance in a happy or beneficial way.",
        "agent": "A system that perceives its environment and takes actions to achieve goals."
    }
    return definitions.get(word.lower(), f"Definition for '{word}' not found in local dictionary.")

In [4]:
@tool
def web_search_tool(query: str) -> str:
    """
    Search the web for information using DuckDuckGo.
    Use this for current events, news, or general knowledge not in your training data.
    """
    try:
        results = DDGS().text(keywords=query, max_results=3)
        if results:
            return "\n".join([f"- {r['title']}: {r['body']}" for r in results])
        return "No results found."
    except Exception as e:
        return f"Search failed: {str(e)}"

In [5]:
# List of tools
tools = [weather_tool, dictionary_tool, web_search_tool]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Define Assistant Node
def assistant(state: MessagesState) -> dict:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [6]:
def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    
    # If the LLM wants to call a tool, route to "tools" node
    if last_message.tool_calls:
        return "tools"
    
    # Otherwise, end the workflow
    return END

In [7]:
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools)) # ToolNode handles execution automatically

# Add edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", should_continue, ["tools", END])
builder.add_edge("tools", "assistant") # Loop back to assistant after using a tool

# Compile with memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agent created with Weather, Dictionary, and Web Search tools.")

✅ Agent created with Weather, Dictionary, and Web Search tools.


In [8]:
def run_query(query, thread_id):
    print(f"\nQUERY: {query}")
    config = {"configurable": {"thread_id": thread_id}}
    result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
    
    for msg in result["messages"]:
        if msg.type == "ai" and msg.tool_calls:
            print(f"🤖 Action: Calling Tool -> {msg.tool_calls[0]['name']}")
        elif msg.type == "tool":
            print(f"🔧 Tool Output: {msg.content[:100]}...") # Truncated for display
        elif msg.type == "ai":
            print(f"🤖 Agent: {msg.content}")

# Test 1: Weather Tool
run_query("What's the weather in Lagos?", "thread_1")

# Test 2: Dictionary Tool
run_query("Define the word 'ephemeral'", "thread_2")

# Test 3: Web Search Tool
run_query("Search for latest AI news", "thread_3")

# Test 4: No Tool Needed
run_query("What's the capital of France?", "thread_4")


QUERY: What's the weather in Lagos?
🤖 Action: Calling Tool -> weather_tool
🔧 Tool Output: Lagos: 32°C, Sunny with high humidity....
🤖 Agent: The weather in Lagos is currently 32°C, sunny, with high humidity.

QUERY: Define the word 'ephemeral'
🤖 Action: Calling Tool -> dictionary_tool
🔧 Tool Output: Lasting for a very short time....
🤖 Agent: The word "ephemeral" means lasting for a very short time.

QUERY: Search for latest AI news


C:\Users\user\AppData\Local\Temp\ipykernel_13916\865602774.py:8: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(keywords=query, max_results=3)


🤖 Action: Calling Tool -> web_search_tool
🔧 Tool Output: - Artificial intelligence | MIT News | Massachusetts Institute of …: Dec 22, 2025 · A “scientific sa...
🤖 Agent: Here are some of the latest news articles related to artificial intelligence:

1. **A “scientific sandbox” lets researchers explore the evolution of vision systems** - This AI-powered tool could inform the design of better sensors and cameras for robots. (Published on Dec 22, 2025, by MIT News)

2. **A new way to increase the capabilities of large language models** - Researchers from the MIT-IBM Watson AI Lab have developed an expressive architecture that provides better state tracking for large language models. (Published on Dec 18, 2025, by MIT News)

3. **Discussion on the factors influencing the global economy, including AI** - Professor Daron Acemoglu speaks with a New York Times reporter about the impact of the artificial intelligence revolution on the global economy. (Published on Dec 19, 2025, by MIT News)

If 